<a href="https://colab.research.google.com/github/sumitv13/Python_sumit/blob/main/MA_Crossover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.express as px


In [31]:
AAPL = yf.download("AAPL",period="2y",progress=False)
AAPL

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-08,74.290001,76.110001,74.290001,75.797501,74.688080,132079200
2020-01-09,76.809998,77.607498,76.550003,77.407501,76.274513,170108400
2020-01-10,77.650002,78.167503,77.062500,77.582497,76.446953,140644800
2020-01-13,77.910004,79.267502,77.787498,79.239998,78.080185,121532000
2020-01-14,79.175003,79.392502,78.042503,78.169998,77.025841,161954400
...,...,...,...,...,...,...
2022-01-03,177.830002,182.880005,177.710007,182.009995,182.009995,104487900
2022-01-04,182.630005,182.940002,179.119995,179.699997,179.699997,99310400
2022-01-05,179.610001,180.169998,174.639999,174.919998,174.919998,94537600


In [32]:
fig = px.line(AAPL, y="Adj Close", title='AAPL Stock Price', labels = {'Adj Close':'AAPL Close Price(in USD)'})
fig.show()

Moving Average 1 (Shorter window)
Here I am choosing Exponential moving average instead of Simple Moving Average, feel free to change it to SMA instead of EMA, you can do so in the following way.

In [33]:
#ema1['Adj Close'] = AAPL['Adj Close'].ewm(span = window1).mean()
window1 = 30
sma1 = pd.DataFrame()
sma1['Adj Close'] = AAPL['Adj Close'].rolling(window = window1).mean()
sma1

,Adj Close
Date,
2020-01-08,NaN
2020-01-09,NaN
2020-01-10,NaN
2020-01-13,NaN
2020-01-14,NaN
...,...
2022-01-03,170.914667
2022-01-04,171.553000
2022-01-05,172.016334


Moving Average 2 (Longer Window)


In [34]:
window2 = 100
sma2 = pd.DataFrame()
sma2['Adj Close'] = AAPL['Adj Close'].rolling(window = window2).mean()
sma2

,Adj Close
Date,
2020-01-08,NaN
2020-01-09,NaN
2020-01-10,NaN
2020-01-13,NaN
2020-01-14,NaN
...,...
2022-01-03,154.872251
2022-01-04,155.182521
2022-01-05,155.442894


In [35]:
fig.add_scatter(x=sma1.index,y=sma1['Adj Close'], mode='lines',name='SMA'+str(window1))
fig.add_scatter(x=sma2.index,y=sma2['Adj Close'], mode='lines',name='SMA'+str(window2))
fig.show()

In [36]:
data = pd.DataFrame()
data['AAPL'] = AAPL['Adj Close']
data['SMA'+str(window1)] = sma1['Adj Close']
data['SMA'+str(window2)] = sma2['Adj Close']
data

,AAPL,SMA30,SMA100
Date,,,
2020-01-08,74.688080,NaN,NaN
2020-01-09,76.274513,NaN,NaN
2020-01-10,76.446953,NaN,NaN
2020-01-13,78.080185,NaN,NaN
2020-01-14,77.025841,NaN,NaN
...,...,...,...
2022-01-03,182.009995,170.914667,154.872251
2022-01-04,179.699997,171.553000,155.182521
2022-01-05,174.919998,172.016334,155.442894


Strategy to generate buy/sell signal

In [37]:
def dualMACrossover(data):
    sigPriceBuy = []
    sigPriceSell = []
    flag = -1 # Flag denoting when the 2 moving averages crossed each other
    for i in range(len(data)):
        if data['SMA'+str(window1)][i] > data['SMA'+str(window2)][i]:
            if flag != 1:
                sigPriceBuy.append(data['AAPL'][i])
                sigPriceSell.append(np.nan)
                flag = 1
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        elif data['SMA'+str(window1)][i] < data['SMA'+str(window2)][i]:
            if flag!=0:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(data['AAPL'][i])
                flag=0
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        else:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
    return (sigPriceBuy,sigPriceSell)

In [38]:
buy_sell = dualMACrossover(data)
data['BuySignalPrice'] = buy_sell[0]
data['SellSignalPrice'] = buy_sell[1]

In [39]:
import plotly.graph_objects as go

fig = px.line(data, y="AAPL", title='Strategy Visualization', labels = {'index':'Date'})
fig.add_scatter(x=data.index,y=data['SMA'+str(window1)], mode='lines',name='SMA'+str(window1))
fig.add_scatter(x=data.index,y=data['SMA'+str(window2)], mode='lines',name='SMA'+str(window2))

fig.add_trace(go.Scatter(mode="markers", x=data.index, y=data.BuySignalPrice, marker_symbol='triangle-up',
                           marker_line_color="#000000", marker_color="#000000", 
                           marker_line_width=2, marker_size=15, name='Buy'))

fig.add_trace(go.Scatter(mode="markers", x=data.index, y=data.SellSignalPrice, marker_symbol='triangle-down',
                           marker_line_color="#E74C3C", marker_color="#E74C3C", 
                           marker_line_width=2, marker_size=15, name='Sell'))
fig.show()

Backtest the strategy


In [40]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

In [41]:
class DualMACrossover(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, window1)
        self.ma2 = self.I(SMA, price, window2)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(AAPL, DualMACrossover,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

Row(id='2149', ...)

In [42]:
stats

Start                     2020-01-08 00:00:00
End                       2022-01-07 00:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                     39.9209
Equity Final [$]                      10944.8
Equity Peak [$]                       11519.4
Return [%]                             9.4481
Buy & Hold Return [%]                 127.145
Return (Ann.) [%]                     4.59878
Volatility (Ann.) [%]                 15.1179
Sharpe Ratio                         0.304194
Sortino Ratio                        0.467771
Calmar Ratio                         0.258577
Max. Drawdown [%]                    -17.7849
Avg. Drawdown [%]                     -4.5904
Max. Drawdown Duration      239 days 00:00:00
Avg. Drawdown Duration       36 days 00:00:00
# Trades                                    4
Win Rate [%]                               25
Best Trade [%]                         29.593
Worst Trade [%]                      -7.34632
Avg. Trade [%]                    